# Langchaing Agents
The following code allows us to use conversation models with langchain, using agents


In [1]:
import os
import openai
import pandas as pd
    
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
data = pd.read_json('info.json')
data.head(5)

,product,review
0,SuperTech Blender,This blender is amazing! It blends everything ...
1,EcoClean Dishwasher,I'm quite disappointed with this dishwasher. I...
2,GourmetChef Knife Set,I absolutely love this knife set. The blades a...
3,CozyDreams Weighted Blanket,This weighted blanket is so comforting. It hel...
4,TechGizmo Smart Watch,"The smartwatch has a lot of potential, but the..."


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [6]:
!pip install -U wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=a935b919e80b3bf14eb91ed99c9d9c09a4a5f37d0eda06e88b5c5b1f1b27b485
  Stored in directory: c:\users\julián ríos\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [7]:
# Importing needed agents libraries
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [8]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [9]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [10]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We need to calculate 25% of 300, which means we need to multiply 300 by 0.25.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:The calculator tool returned the answer 75.0, which is correct.

Final Answer: 25% of 300 is 75.0.

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '25% of 300 is 75.0.'}

In [11]:
# Python Agent
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [12]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [13]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

# I could set langchaing.debug = True to see all the process



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python_REPL
Action Input: 
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted(customers, key=lambda x: (x[1], x[0]))
```
Observation: 
Thought:The output is a sorted list of customers by last name and then first name.
Action: Python_REPL
Action Input: `print(sorted(customers, key=lambda x: (x[1], x[0])))`
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.

"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [14]:
# Customize own agents
from langchain.agents import tool
from datetime import date

In [15]:
@tool # Using decorators
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [16]:
agent= initialize_agent(
    tools + [time], # Adding "time" tool
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [17]:
agent.run("whats the date today")
# I can use try except for this



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-08-28
Thought:I have successfully retrieved today's date using the `time` tool.
Final Answer: Today's date is August 28th, 2023.

> Finished chain.


"Today's date is August 28th, 2023."